In [1]:
!pip install tensorflow==2.15.0
!pip install numpy==1.24.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.9 MB/s eta 0:00:0000:010:01
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 76.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0

In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

2024-07-02 10:49:40.593758: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 10:49:40.593900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 10:49:40.713573: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Define the number of classes (e.g., 12 hours * 60 minutes)
num_classes = 12 * 60

In [4]:
# Load and preprocess dataset
dataset_path = '/kaggle/input/simple-analog-clock-monochrome/clock_minute_hand/samples/'
filenames = os.listdir(dataset_path)

# Process labels to remove image extensions and convert them to one-hot encoded format
labels = {}
for filename in filenames:
    # Remove the '.png' extension and split by '_'
    time_parts = filename.replace('.png', '').split('_')
    # Convert to numerical format (e.g., minutes past midnight)
    hour = int(time_parts[1])
    if hour == 12:
        hour = 0  # Set 12 to 0
    minute = int(time_parts[2])
    total_minutes = hour * 60 + minute
    # One-hot encode the label
    one_hot_label = to_categorical(total_minutes, num_classes)
    labels[filename] = one_hot_label

# Read images and convert labels
images = []
processed_labels = []
for filename in filenames:
    # Read and preprocess the image
    img = cv2.imread(os.path.join(dataset_path, filename), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224, 224))  # Resize to match the input shape of the model
    img = img / 255.0  # Normalize pixel values
    images.append(img)
    
    # Process and store the label
    processed_labels.append(labels[filename])

# Convert lists to numpy arrays
images = np.array(images).reshape(-1, 224, 224, 1)  # Add the channel dimension for grayscale
processed_labels = np.array(processed_labels)

In [5]:
processed_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
# Model architecture
inputs = Input(shape=(224, 224, 1))

x = Conv2D(filters = 256, kernel_size = (3,3), padding = 'valid', activation = 'relu')(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size = (2,2), padding = "valid")(x)


x = Conv2D(filters = 256, kernel_size = (3,3), padding = 'valid', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size = (2,2), padding = "valid")(x)


x = Conv2D(filters = 128, kernel_size = (3,3), padding = 'valid', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size = (2,2), padding = "valid")(x)


x = Conv2D(filters = 128, kernel_size = (3,3), padding = 'valid', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size = (2,2), padding = "valid")(x)


x = Conv2D(filters = 256, kernel_size = (3,3), padding = 'valid', activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size = (2,2), padding = "valid")(x)


x = Flatten()(x)

x = Dense(units = 128, activation = 'relu')(x)
x = Dropout(0.1)(x)

x = Dense(units = 128, activation = 'relu')(x)
x = Dropout(0.1)(x)

x = Dense(units = 64, activation = 'relu')(x)
x = Dropout(0.1)(x)

x = Dense(units = 32, activation = 'relu')(x)
x = Dropout(0.1)(x)

# Adjust the last Dense layer to match the number of classes
x = Dense(units=num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=inputs, outputs=x)

# Compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ["accuracy"])

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 256)     2560      
                                                                 
 batch_normalization (Batch  (None, 222, 222, 256)     1024      
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 256)     0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 256)     590080    
                                                                 
 batch_normalization_1 (Bat  (None, 109, 109, 256)     1024  

In [8]:
model_name = "AnalogClockTimeRecModel.h5"
checkpoint = ModelCheckpoint(model_name, monitor = "val_loss", mode = "min", save_best_only = True, verbose = 1)

calls = EarlyStopping(patience = 5, verbose = True)

In [ ]:
history = model.fit(images, processed_labels, epochs = 100, validation_split = 0.2, callbacks = [checkpoint, calls], verbose = 1)

Epoch 1/100


I0000 00:00:1719917456.045854     113 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


250/250 [==============================] - ETA: 0s - loss: 6.2632 - accuracy: 0.0075
Epoch 1: val_loss improved from inf to 6.53222, saving model to AnalogClockTimeRecModel.h5
250/250 [==============================] - 68s 211ms/step - loss: 6.2632 - accuracy: 0.0075 - val_loss: 6.5322 - val_accuracy: 0.0015
Epoch 2/100


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


250/250 [==============================] - ETA: 0s - loss: 4.7453 - accuracy: 0.0457
Epoch 2: val_loss improved from 6.53222 to 4.18268, saving model to AnalogClockTimeRecModel.h5
250/250 [==============================] - 50s 202ms/step - loss: 4.7453 - accuracy: 0.0457 - val_loss: 4.1827 - val_accuracy: 0.1125
Epoch 3/100
250/250 [==============================] - ETA: 0s - loss: 3.9199 - accuracy: 0.0854
Epoch 3: val_loss improved from 4.18268 to 2.92665, saving model to AnalogClockTimeRecModel.h5
250/250 [==============================] - 50s 202ms/step - loss: 3.9199 - accuracy: 0.0854 - val_loss: 2.9266 - val_accuracy: 0.2115
Epoch 4/100
250/250 [==============================] - ETA: 0s - loss: 3.4489 - accuracy: 0.1310
Epoch 4: val_loss improved from 2.92665 to 2.46100, saving model to AnalogClockTimeRecModel.h5
250/250 [==============================] - 50s 202ms/step - loss: 3.4489 - accuracy: 0.1310 - val_loss: 2.4610 - val_accuracy: 0.2760
Epoch 5/100
250/250 [=============

In [ ]:
# Plot training history
plt.figure(figsize=(12, 6))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## Manual Test

In [ ]:
# Function to preprocess the image
def preprocess_image(image_path, img_height=224, img_width=224):
    # Read and preprocess the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img_height, img_width))  # Resize to match the input shape of the model
    img = img / 255.0  # Normalize pixel values
    img = np.array(img).reshape(-1, img_height, img_width, 1)  # Add the channel dimension for grayscale
    return img

In [ ]:
# Load the trained model
model = load_model('AnalogClockTimeRecModel.keras')

# Path to your new image
new_image_path = r'/kaggle/input/clockface-handwritten/fc3.png'

# Preprocess the image
preprocessed_image = preprocess_image(new_image_path)

# Get the prediction
prediction = model.predict(preprocessed_image)

# Convert the prediction to the original label
predicted_minutes = np.argmax(prediction)
predicted_hour = predicted_minutes // 60
predicted_minute = predicted_minutes % 60

# Adjust the hour if it was set to 0 for 12 o'clock
if predicted_hour == 0:
    predicted_hour = 12

print(f"The predicted time is {predicted_hour}:{predicted_minute:02d}")

In [ ]:
# Get the prediction
prediction = model.predict(preprocessed_image)

# Get the indices of the top 10 predictions
top_10_indices = np.argsort(prediction[0])[-10:][::-1]

# Get the top 10 probabilities
top_10_probabilities = prediction[0][top_10_indices]

# Convert the indices to the original labels (time)
top_10_predictions = [(index // 60, index % 60) for index in top_10_indices]

# Adjust the hours if they were set to 0 for 12 o'clock
top_10_predictions = [(12 if hour == 0 else hour, minute) for hour, minute in top_10_predictions]

# Print the top 10 predictions with their probabilities
for i, (hour, minute) in enumerate(top_10_predictions):
    print(f"Prediction {i+1}: {hour}:{minute:02d} with probability {top_10_probabilities[i]:.10f}")